In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rtb.datasets import get_dataset
from rtb.utils import rolling_window_sampler, one_window_sampler

In [3]:
ds = get_dataset(
    name="rtb-product",
    root="../../data",
    # default is books-5-core
    # category="fashion",
    # use_5_core=False,
    # process=True,
)

100%|██████████████████████████████████████████████████████████████| 7.10G/7.10G [01:44<00:00, 67.7MiB/s]


loading table ../../data/rtb-product-books_5_core/processed/db/customer.parquet...
done in 0.57 seconds.
loading table ../../data/rtb-product-books_5_core/processed/db/product.parquet...
done in 1.49 seconds.
loading table ../../data/rtb-product-books_5_core/processed/db/review.parquet...
done in 44.43 seconds.


In [4]:
db = ds._db

In [5]:
customer = db.tables["customer"].df
product = db.tables["product"].df
review = db.tables["review"].df

In [6]:
db.tables["customer"].df

,customer_id,customer_name
0,0,TW Ervin II
1,1,Amazon Customer
2,2,Tekla Borner
3,3,Deborah K Woroniecki
4,4,E
...,...,...
1850188,1850188,Amazon Customer
1850189,1850189,AmizoP
1850190,1850190,JuicyBeans
1850191,1850191,Halley Cortez


In [7]:
product

,product_id,category,brand,title,description,price
0,0,"[Books, Literature & Fiction, Dramas & Plays]",Visit Amazon's William Shakespeare Page,Love's Labour's Lost: Performed by Derek Jacob...,William Shakespeare is widely regarded as the ...,20.93
1,1,"[Books, Children's Books, Literature & Fiction]",Louis (ed) UNTERMEYER,The golden treasury of poetry,Collection of Poetry,23.98
2,2,"[Books, Mystery, Thriller & Suspense]",Visit Amazon's Agatha Christie Page,The Blue Geranium (Miss Marple),None,8.82
3,3,"[Books, Humor & Entertainment]",Visit Amazon's Tom Baker Page,Who on Earth Is Tom Baker?,Tom Baker's autobiography covers his childhood...,159.95
4,4,"[Books, Christian Books & Bibles, Bible Study ...",Visit Amazon's Clarence Larkin Page,The Greatest Book on &quot;Dispensational Trut...,The Greatest Book on Dispensational Truth in t...,44.76
...,...,...,...,...,...,...
506007,506007,None,"Tricia & Putman, Cara Goyer",Mason Jar Mayhem,"Spring has come to Sugarcreek, and the communi...",15.05
506008,506008,None,Author,The Little Book of Pussy (2013-06-16) [Paperback],From the time The Big Penis Book was published...,17.00
506009,506009,None,,The Age of Not Believing: A Christmas Tale by ...,None,42.93
506010,506010,None,,The Charlies Angels' Casebook,None,17.87


In [8]:
review

,review_time,customer_id,product_id,rating,verified,review_text,summary
0,2005-03-30,0,14,5.0,False,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from
1,2016-06-20,1,14,5.0,True,The kids loved it!,Five Stars
2,2016-01-24,2,14,5.0,True,My students (3 & 4 year olds) loved this book!...,Five Stars
3,2015-07-09,3,14,5.0,False,LOVE IT,Five Stars
4,2015-01-18,4,14,5.0,True,Great!,Five Stars
...,...,...,...,...,...,...,...
21935279,2012-12-02,319196,506010,5.0,False,"This past year fans of ""Charlie's Angels"" were...","""NOW THIS IS THE COMPANION TO THE ""CHARLIE'S A..."
21935280,2008-12-15,1407251,506010,5.0,True,"For any fan of Charlie's Angels, this book is ...",An Excellent Case!
21935281,2007-06-15,1373107,506010,3.0,False,To review this book I will divide it into two ...,Great Stories But A Little One-Sided
21935282,2018-05-27,319324,506011,3.0,True,"Great pictures, good recommendations but not t...",Don't buy if your looking for details and sett...


## LTV

In [9]:
window_size = ds.tasks["ltv"].window_sizes[0]
window_size

Timedelta('730 days 00:00:00')

### train table

In [10]:
# default time window sampler used inside make_train_table
time_window_df = rolling_window_sampler(
    ds.min_time, ds.train_max_time, window_size, window_size
)
time_window_df

,window_min_time,window_max_time
0,2011-01-02,2013-01-01
1,2009-01-02,2011-01-02
2,2007-01-03,2009-01-02
3,2005-01-03,2007-01-03
4,2003-01-04,2005-01-03
5,2001-01-04,2003-01-04
6,1999-01-05,2001-01-04


In [11]:
train_table = ds.make_train_table("ltv", window_size)

In [12]:
# here count is the number of products which contributed to the sum aggregation in ltv
train_table.df

,window_min_time,window_max_time,customer_id,ltv,count_
0,2011-01-02,2013-01-01,1843206,30.95,2
1,2011-01-02,2013-01-01,1843212,0.00,0
2,2011-01-02,2013-01-01,1843346,28.48,1
3,2011-01-02,2013-01-01,1843357,0.00,0
4,2011-01-02,2013-01-01,1843368,0.00,0
...,...,...,...,...,...
573737,1999-01-05,2001-01-04,1227678,0.00,0
573738,2011-01-02,2013-01-01,1227883,0.00,0
573739,2011-01-02,2013-01-01,1228351,0.00,0
573740,2009-01-02,2011-01-02,1228209,37.59,2


In [13]:
# frequency of counts
train_table.df["count_"].value_counts(normalize=True).sort_index()

count_
0       0.517581
1       0.186038
2       0.096230
3       0.054913
4       0.033184
          ...   
996     0.000002
1088    0.000002
1114    0.000002
1130    0.000002
1541    0.000002
Name: proportion, Length: 377, dtype: float64

## val table

In [14]:
# time sampler used inside make_val_table
time_window_df = one_window_sampler(ds.val_max_time, window_size)
time_window_df

,window_min_time,window_max_time
0,2015-01-01,2016-12-31


In [15]:
val_table = ds.make_val_table("ltv", window_size)

In [16]:
val_table.df

,window_min_time,window_max_time,customer_id,ltv,count_
0,2013-01-01,2015-01-01,307203,37.96,2
1,2013-01-01,2015-01-01,307206,0.00,0
2,2013-01-01,2015-01-01,307209,1311.63,104
3,2013-01-01,2015-01-01,307219,64.76,3
4,2013-01-01,2015-01-01,307220,0.00,0
...,...,...,...,...,...
441579,2013-01-01,2015-01-01,1839020,0.00,0
441580,2013-01-01,2015-01-01,1837576,0.00,0
441581,2013-01-01,2015-01-01,1837770,0.00,0
441582,2013-01-01,2015-01-01,1837854,0.00,0


In [17]:
val_table.df["count_"].value_counts(normalize=True).sort_index()

count_
0       0.234166
1       0.165880
2       0.134926
3       0.104857
4       0.076488
          ...   
1055    0.000002
1171    0.000002
1400    0.000002
1778    0.000002
2038    0.000002
Name: proportion, Length: 403, dtype: float64

## Churn

In [18]:
window_size = ds.tasks["churn"].window_sizes[0]
window_size

Timedelta('730 days 00:00:00')

### train table

In [19]:
# default time window sampler used inside make_train_table
time_window_df = rolling_window_sampler(
    ds.min_time, ds.train_max_time, window_size, window_size
)
time_window_df

,window_min_time,window_max_time
0,2011-01-02,2013-01-01
1,2009-01-02,2011-01-02
2,2007-01-03,2009-01-02
3,2005-01-03,2007-01-03
4,2003-01-04,2005-01-03
5,2001-01-04,2003-01-04
6,1999-01-05,2001-01-04


In [20]:
train_table = ds.make_train_table("churn", window_size)

In [21]:
train_table.df

,window_min_time,window_max_time,customer_id,churn
0,2011-01-02,2013-01-01,1843206,0
1,2011-01-02,2013-01-01,1843212,1
2,2011-01-02,2013-01-01,1843341,1
3,2011-01-02,2013-01-01,1843344,1
4,2011-01-02,2013-01-01,1843346,0
...,...,...,...,...
573737,2001-01-04,2003-01-04,1330343,1
573738,2001-01-04,2003-01-04,1330450,1
573739,2001-01-04,2003-01-04,1330457,1
573740,2001-01-04,2003-01-04,1331029,1


In [22]:
# frequency of churn
train_table.df["churn"].value_counts(normalize=True)

churn
1    0.517581
0    0.482419
Name: proportion, dtype: float64

## val table

In [23]:
# time sampler used inside make_val_table
time_window_df = one_window_sampler(ds.train_max_time, window_size)
time_window_df

,window_min_time,window_max_time
0,2013-01-01,2015-01-01


In [24]:
val_table = ds.make_val_table("churn", window_size)

In [25]:
val_table.df

,window_min_time,window_max_time,customer_id,churn
0,2013-01-01,2015-01-01,1843206,0
1,2013-01-01,2015-01-01,1843211,0
2,2013-01-01,2015-01-01,1843213,1
3,2013-01-01,2015-01-01,1843233,1
4,2013-01-01,2015-01-01,1843235,1
...,...,...,...,...
441579,2013-01-01,2015-01-01,1832903,0
441580,2013-01-01,2015-01-01,1832921,0
441581,2013-01-01,2015-01-01,1832923,0
441582,2013-01-01,2015-01-01,1832934,0


In [26]:
val_table.df["churn"].value_counts(normalize=True) * 100

churn
0    76.583391
1    23.416609
Name: proportion, dtype: float64